## Auditing and Exporting Detections

**Before running this script, make sure that your Google Drive folder contains the orthomosaic GeoTiff (`step 0`), the `spatial_data.json` file (`step 1`), the `classes.csv` and `subset_list` files (`step3`) and the `new_detections.json` file (`step 4`).**

<a href="https://colab.research.google.com/github/gl7176/GreySealCNN/blob/master/5_export_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<center> Be sure to update this hyperlink above if you clone and want to point to a different GitHub </center>

### Connect to our Google Drive folder and pull files
Note: when you run this it will give you a link that you must click. You must give Google some permissions, then copy a code into a box that comes up in the output section of this code.

If customizing this code, you will need to point the `drive_folder` variable to a URL for your shared google drive folder.

In [26]:
!pip install -U -q PyDrive
import os, csv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters

# set variable to the destination google drive folder you want to pull from
drive_folder = 'https://drive.google.com/drive/folders/1INuRNVKvKMy8L_Nb6lmoVbyvScWK0-0D'

# this bit points the code to that google drive folder
pointer = str("'" + drive_folder.split("/")[-1] + "'" + " in parents")

file_list = drive.ListFile(
    {'q': pointer}).GetList()

#    this bit pulls key files from the directory specified above
#    and checks that all necessary files are present


for f in file_list:
  # 3. Create & download by id.
  fname = os.path.join(local_download_path, f['title'])
  if fname.endswith(".tif") or fname.endswith(".json") or fname.endswith(".csv"):
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    if fname.endswith(".csv"):
      print("Pulled file: " + fname)

Pulled file: data/annotations_test.csv
Pulled file: data/classes.csv
Pulled file: data/annotations_valid.csv
Pulled file: data/annotations_train.csv
Pulled file: data/subset_list.csv
Pulled file: data/via_SealCNN_TrainingData.csv


### Identify necessary files from the input directory

In [27]:
# use this variable to set input directory
input_dir = local_download_path

orthomosaic_file = 'orthomosaic_placeholder'
spatial_data_file = 'spatial_data.json'
classes_file = 'classes.csv'
new_detections_file = 'new_detections.json'
subset_list_file = 'subset_list.csv'
annotations_file = 'annotations_placeholder'

checklist = {orthomosaic_file:"orthomosaic_file", spatial_data_file:"spatial_data_file",
             classes_file:"classes_file", new_detections_file:"new_detections_file",
             subset_list_file:"subset_list_file", annotations_file:"annotations_file"}
 
for fname in os.listdir(input_dir):
    candidate_file = "{i}/{f}".format(i=input_dir, f=fname)
    os.stat(candidate_file)
    # if the file is a *.tif and larger than 100 mb we label it the orthomosaic
    if fname.endswith(".tif") and os.stat(candidate_file).st_size > 10**8 :
      # if there are multiple orthomosaic files detected we spit an error
      if orthomosaic_file != 'orthomosaic_placeholder':
        raise Exception("more than one orthomosaic file identified based on size and type")
      orthomosaic_file = "{i}/{f}".format(i=input_dir, f=fname)
      print("orthomosaic identified as " + orthomosaic_file)
      del checklist['orthomosaic_placeholder']
    elif fname.endswith(".csv") or fname.endswith(".json"):
      with open(candidate_file, "r") as f:
        if next(csv.reader(f, delimiter=","))[0:3] == ['filename', 'file_size', 'file_attributes']:
          annotations_file = candidate_file
          print("annotations file identified as " + annotations_file)
          del checklist['annotations_placeholder']
        else:
          try: 
            vars()[checklist[fname]] = "{i}/{f}".format(i=input_dir, f=fname)
            print("required file found: {v}".format(v=vars()[checklist[fname]]))
            del checklist[fname]
          except: print("{f} detected but not listed among requirements".format(f=fname))

if len(checklist) > 0:
  for key in checklist:
    print("Error: did not find {k} in your input folder".format(k=key))
  raise Exception("missing specified data files")

annotations file identified as data/via_SealCNN_TrainingData.csv
required file found: data/classes.csv
orthomosaic identified as data/2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1.tif
required file found: data/spatial_data.json
required file found: data/new_detections.json
annotations_valid.csv detected but not listed among requirements
annotations_train.csv detected but not listed among requirements
annotations_test.csv detected but not listed among requirements
required file found: data/subset_list.csv


### Set up the python environment and key variables

In [28]:
import os
import argparse
import numpy as np
import json
import csv
!pip install rasterio==1.1.8
import rasterio
import copy
from collections import OrderedDict
!pip install fiona
import fiona # only required for exporting to shapefiles
from fiona.crs import from_epsg
from shapely.geometry import mapping, Polygon

# make a dictionary where {filename} calls {subset type}
with open(subset_list_file, "r") as f:
    reader = csv.reader(f, delimiter=",")
    subset_dict = {i[0]:i[1] for i in reader}

### Create some useful functions and variables

In [29]:
# takes a list and replaces dictionaries of {'box': [x1, y1, x2, y2]} in file-pixel coordinates
# with dictionaries of {'box': array([[ 291942.12379, 5099949.2831 ], [ 291944.92633, 5099949.2831 ],
# [291944.92633, 5099946.87579], [ 291942.12379, 5099946.87579]])} in global coordinates

# set dataset variable to our orthomosaic
dataset = rasterio.open(orthomosaic_file)

# pull labels from classes.csv
import csv
with open(classes_file, "r") as f:
    reader = csv.reader(f, delimiter=",")
    labels_to_names = {int(i[1]):i[0] for i in reader}

# Set output directory, create it if necessary
output_dir = 'shapefile_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# define function to transform coordinates from orthomosaic/pixel reference to global reference
def global_transform(box_list):
    geolocated_bb = []
    for box in box_list:
      bounding_box = np.array([[box['box'][0], box['box'][1]], [box['box'][2], box['box'][1]], [box['box'][2], box['box'][3]], [box['box'][0], box['box'][3]]]).astype(float)
      count = 0
      for point in bounding_box:
        point = dataset.transform * point
        bounding_box[count] = point
        count += 1
      box['box'] = bounding_box
    print(box_list[0:3])


# Malisiewicz et al.
# https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
def non_max_suppression(boxes, probs, overlapThresh):
  # if there are no boxes, return an empty list
  if len(boxes) == 0:
      return []

  # if the bounding boxes are integers, convert them to floats -- this
  # is important since we'll be doing a bunch of divisions
  if boxes.dtype.kind == "i":
      boxes = boxes.astype("float")

  # initialize the list of picked indexes
  pick = []

  # grab the coordinates of the bounding boxes
  x1, y1, x2, y2 = boxes[:, 0], boxes[:, 1], boxes[:, 2], boxes[:, 3]

  # compute the area of the bounding boxes and grab the indexes to sort
  # (in the case that no probabilities are provided, simply sort on the
  # bottom-left y-coordinate)
  area = (x2 - x1 + 1) * (y2 - y1 + 1)
  idxs = y2

  # if probabilities are provided, sort on them instead
  if probs is not None:
      idxs = probs

  # sort the indexes
  idxs = np.argsort(idxs)

  # keep looping while some indexes still remain in the indexes list
  while len(idxs) > 0:
      # grab the last index in the indexes list and add the index value
      # to the list of picked indexes
      last = len(idxs) - 1
      i = idxs[last]
      pick.append(i)

      # find the largest (x, y) coordinates for the start of the bounding
      # box and the smallest (x, y) coordinates for the end of the bounding
      # box
      xx1, yy1, xx2, yy2 = np.maximum(x1[i], x1[idxs[:last]]), np.maximum(y1[i], y1[idxs[:last]]), np.minimum(x2[i], x2[idxs[:last]]), np.minimum(y2[i], y2[idxs[:last]])

      # compute the width and height of the bounding box
      w, h = np.maximum(0, xx2 - xx1 + 1), np.maximum(0, yy2 - yy1 + 1)

      # compute the ratio of overlap
      overlap = (w * h) / area[idxs[:last]]

      # delete all indexes from the index list that have overlap greater
      # than the provided overlap threshold
      idxs = np.delete(idxs, np.concatenate(([last],
          np.where(overlap > overlapThresh)[0])))

  # return the index of the bounding boxes that were picked
  return pick

### Convert detections from image-/tile-based coordinates to orthomosaic coordinates

In [30]:
# open the output from our CNN
with open(new_detections_file) as f:
    detected_labels = json.load(f)

# open the output from our original tile splitting
with open(spatial_data_file) as f:
    spatial_data = json.load(f)
    tile_height = spatial_data["tile_height"]
    tile_width = spatial_data["tile_width"]
    img_data = spatial_data["tile_pointers"]
    
# update box locations from local tile coordinates to orthomosaic coordinates
# using image_locations from original tiling process, invoked by filename
# ---this involves transforming x1, y1, x2, y2 coordinates to 4 sets of x,y points
# and then transforming them back to the original form---
# then build a list of detection dictionaries. Note that filename is no longer
# needed because coordinates are now relative to the orthomosaic, not tile

detection_list = []
for key, value in detected_labels.items():
    for detection in value:
      # convert bounding box from x1/y1/x2/y2 format to [x1,y1],[x2,y1],[x2,y2],[x2,y1] coordinates format 
      bounding_box = np.array([[detection['box'][0], detection['box'][1]], [detection['box'][2], detection['box'][1]], [detection['box'][2], detection['box'][3]], [detection['box'][0], detection['box'][3]]])
      # update the new coordinates format from local tile coordinates to orthomosaic coordinates
      bounding_box = bounding_box + [img_data["image_locations"][[key][0].split("/")[-1]]]
      # convert our bounding box from coordinates format back to x1/y1/x2/y2 format
      bounding_box = [bounding_box[0][0], bounding_box[0][1], bounding_box[1][0], bounding_box[2][1]] 
      # update the dictionary
      detection['box'] = bounding_box
      #build our detection list. Note we no longer need filenames because the coordinates are no longer local
      detection_list.append(detection)
print(detection_list[0:3])

[{'box': [9124, 9845, 9168, 9880], 'label': 1, 'score': 0.9468869566917419}, {'box': [9085, 9894, 9123, 9931], 'label': 1, 'score': 0.9024936556816101}, {'box': [8428, 9849, 8482, 9925], 'label': 0, 'score': 0.8625706434249878}]


### Implement non-max suppression on duplicate CNN detections

In [31]:
boxes = []
scores = []
for detection in detection_list:
  boxes.append(detection['box'])
  scores.append(detection['score'])
  
bboxes = np.array(boxes)
pick = non_max_suppression(bboxes, scores, 0.6)
nms_detection_list = []
for i in pick:
  nms_detection_list.append(detection_list[i])

# backup the nms_detection_list before running next section, so if something
# goes wrong we don't need to re-run the whole code
nms_detection_list_backup = copy.deepcopy(nms_detection_list)

print("Before NMS: " + str(len(detection_list)) + " detections")
print("After NMS: " + str(len(nms_detection_list)) + " detections")

Before NMS: 189 detections
After NMS: 186 detections


### Export CNN detections shapefile

In [32]:
# the global transform function permanently alters nms_detection_list, necessitating the deepcopy backup earlier
nms_detection_list = copy.deepcopy(nms_detection_list_backup)
global_transform(nms_detection_list)

[{'box': array([[ 292165.68025, 5099999.2258 ],
       [ 292167.26117, 5099999.2258 ],
       [ 292167.26117, 5099997.96825],
       [ 292165.68025, 5099997.96825]]), 'label': 1, 'score': 0.9468869566917419}, {'box': array([[ 292081.46033, 5100142.90987],
       [ 292084.08322, 5100142.90987],
       [ 292084.08322, 5100141.1493 ],
       [ 292081.46033, 5100141.1493 ]]), 'label': 0, 'score': 0.9271194934844971}, {'box': array([[ 292341.27016, 5100226.51898],
       [ 292343.13852, 5100226.51898],
       [ 292343.13852, 5100224.03981],
       [ 292341.27016, 5100224.03981]]), 'label': 0, 'score': 0.9254277348518372}]


In [33]:
# Define your schema as a polygon geom with a couple of fields
# then write out the detections as a shapefile
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str'),
        ('Score', 'float')
  ])
}

with fiona.open(output_dir + '/seal_detections.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(nms_detection_list):
      record = {
            'geometry': {'coordinates': [polygon['box']], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', orthomosaic_file),
                                       ('Detection', labels_to_names[polygon['label']]),
                                       ('Score', polygon['score'])
                                       ]),
            'type': 'Feature'}
      c.write(record)

#Exporting original annotations

### Format original VIA annotations to necessary information

In [34]:
via_annotations_list = []

# read each line, parse it, convert it, put it all back together
# then drop it in the appropriate subset
with open(annotations_file, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader: 
        # output we want:
        # format: path/to/image.jpg,x1,y1,x2,y2,class_name
        # example: /data/imgs/img_001.jpg,837,346,981,456,cow
        if 'filename' in line[0]:
            # bypassing comments in csv
            continue
        if '{}' in line[5]:
            #bypassing empty images
            continue
            
        filename = line[0]
        
        # pulling from column named "region_shape_attributes"
        box_entry = json.loads(line[5])
        top_left_x, top_left_y, width, height = box_entry["x"], box_entry["y"], box_entry["width"], box_entry["height"]
        if width == 0 or height == 0:
            continue
            # skip tiny/empty boxes
        
        # define area for NMS ranking later
        area = width * height

        # convert from "top left and width/height" to "x and y values at each corner of the box"
        if top_left_x < 0: top_left_x = 1
        if top_left_y < 0: top_left_y = 1
        x1, x2, y1, y2 = top_left_x, top_left_x + width, top_left_y, top_left_y + height 
        
        # pulling from column named "region_attributes" to get class names
        name = json.loads(line[6])["Age Class"]

        # skip unknown class, in this case. Might be useful in other applications though, e.g. total count
        if name == "Unknown":
            continue
        
        # pull subset from dictionary
        subset_type = subset_dict[filename]

        # create the annotation row
        new_row = [filename, [x1,y1], [x2,y1], [x2,y2], [x1,y2], name, area, subset_type]

        # append the row to the our list
        via_annotations_list.append(new_row)
print(via_annotations_list[0:5])

[['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png', [615, 927], [681, 927], [681, 959], [615, 959], 'Adult', 2112, 'training'], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png', [959, 917], [998, 917], [998, 943], [959, 943], 'Pup', 1014, 'training'], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png', [632, 899], [668, 899], [668, 932], [632, 932], 'Pup', 1188, 'training'], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---28.png', [514, 937], [581, 937], [581, 968], [514, 968], 'Adult', 2077, 'training'], ['2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---29.png', [152, 671], [192, 671], [192, 697], [152, 697], 'Pup', 1040, 'training']]


### Convert via annotations from image-pixel coordinates to orthomosaic-pixel coordinates

In [35]:
# give each detection a score for NMS, based on its area and prioritized by
# testing > training datasets so during NMS if we encounter duplicates we prioritize
# testing data for accuracy evaluation, and within that prioritization, we prioritize
# larger boxes over smaller boxes (in case any "edge" animals were duplicated)
area_max = max(list(i[6] for i in via_annotations_list))
scores = []
boxes = []

detection_list = []
for i in via_annotations_list:
  score = i[6]/area_max
  if i[7] != 'testing':
    score = 0.01 * score
  scores.append(score)

  bounding_box = np.array([i[1],i[2],i[3],i[4]])
  # update the new coordinates format from local tile coordinates to orthomosaic coordinates
  bounding_box = bounding_box + [img_data["image_locations"][i[0]]]
  # convert our bounding box from coordinates format back to x1/y1/x2/y2 format
  bounding_box = [bounding_box[0][0], bounding_box[0][1], bounding_box[1][0], bounding_box[2][1]] 
  boxes.append(bounding_box)

  # update the dictionary
  detection = {"box":bounding_box, "score":score, "label":i[5], "subset":i[7]}
  detection_list.append(detection)
print(detection_list[0:3])

[{'box': [9815, 1847, 9881, 1879], 'score': 0.003300515705578997, 'label': 'Adult', 'subset': 'training'}, {'box': [10159, 1837, 10198, 1863], 'score': 0.001584622597280825, 'label': 'Pup', 'subset': 'training'}, {'box': [9832, 1819, 9868, 1852], 'score': 0.0018565400843881857, 'label': 'Pup', 'subset': 'training'}]


### Implement NMS on orthomosaic

In [36]:
bboxes = np.array(boxes)
pick = non_max_suppression(bboxes, scores, 0.6)
nms_detection_list = []
for i in pick:
  nms_detection_list.append(detection_list[i])

# backup the nms_detection_list before running next section, so if something
# goes wrong we don't need to re-run the whole code
nms_detection_list_backup = copy.deepcopy(nms_detection_list)

print("Before NMS: " + str(len(detection_list)) + " detections")
print("After NMS: " + str(len(nms_detection_list)) + " detections")

Before NMS: 2719 detections
After NMS: 2371 detections


### Export VIA annotations shapefile

In [37]:
nms_detection_list = copy.deepcopy(nms_detection_list_backup)
global_transform(nms_detection_list)

[{'box': array([[ 292015.24134, 5100059.30076],
       [ 292018.22353, 5100059.30076],
       [ 292018.22353, 5100056.85752],
       [ 292015.24134, 5100056.85752]]), 'score': 0.882012814502266, 'label': 'Adult', 'subset': 'testing'}, {'box': array([[ 292011.57648, 5100085.35001],
       [ 292013.55263, 5100085.35001],
       [ 292013.55263, 5100082.18817],
       [ 292011.57648, 5100082.18817]]), 'score': 0.7563681825285201, 'label': 'Adult', 'subset': 'testing'}, {'box': array([[ 291923.83542, 5099943.5343 ],
       [ 291926.27866, 5099943.5343 ],
       [ 291926.27866, 5099940.98327],
       [ 291923.83542, 5099940.98327]]), 'score': 0.7544928895139865, 'label': 'Adult', 'subset': 'testing'}]


In [38]:
# write out the detections as a shapefile
# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Detection', 'str'),
        ('Subset', 'str'),
  ])
}

with fiona.open(output_dir + '/via_annotations.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(nms_detection_list):
      record = {
            'geometry': {'coordinates': [polygon['box']], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', orthomosaic_file),
                                       ('Detection', polygon['label']),
                                       ('Subset', polygon['subset'])
                                       ]),
            'type': 'Feature'}
      c.write(record)

# Exporting subset regions

### Convert tile spatial data to boxes

In [39]:
tile_list = []
for i in img_data["image_locations"]:
  x1, y1 = img_data["image_locations"][i]
  x2, y2 = x1 + tile_width, y1 + tile_height
  tile_square = {'box': [x1, y1, x2, y2], 'file_name':i, 'subset':subset_dict[i]}
  tile_list.append(tile_square)
global_transform(tile_list)

# get it to x1/y1/x2/y2 format

# takes a list and replaces dictionaries of {'box': [x1, y1, x2, y2]} in file-pixel coordinates
# with dictionaries of {'box': array([[ 291942.12379, 5099949.2831 ], [ 291944.92633, 5099949.2831 ],
# [291944.92633, 5099946.87579], [ 291942.12379, 5099946.87579]])} in global coordinates

# def global_transform(box_list, geo_reference_file):

[{'box': array([[ 292135.35533, 5100352.95665],
       [ 292171.28533, 5100352.95665],
       [ 292171.28533, 5100317.02665],
       [ 292135.35533, 5100317.02665]]), 'file_name': '2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---9.png', 'subset': 'training'}, {'box': array([[ 292168.41093, 5100352.95665],
       [ 292204.34093, 5100352.95665],
       [ 292204.34093, 5100317.02665],
       [ 292168.41093, 5100317.02665]]), 'file_name': '2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---10.png', 'subset': 'training'}, {'box': array([[ 292201.46653, 5100352.95665],
       [ 292237.39653, 5100352.95665],
       [ 292237.39653, 5100317.02665],
       [ 292201.46653, 5100317.02665]]), 'file_name': '2015_02_02_hay_island_flight03_s110rgb_jpeg_mosaic_group1---11.png', 'subset': 'training'}]


Export tile footprints as shapefile

In [40]:
# Define your schema as a polygon geom with a couple of fields
schema = {
    'geometry': 'Polygon',
    'properties': OrderedDict([
        ('ImageName', 'str'),
        ('Subset', 'str'),
  ])
}

with fiona.open(output_dir + '/tile_footprints.shp',
    'w',
    driver='ESRI Shapefile',
    crs=dataset.crs,
    schema=schema) as c:
    
    for num, polygon in enumerate(tile_list):
      record = {
            'geometry': {'coordinates': [polygon['box']], 'type': 'Polygon'},
            'id': num,
            'properties': OrderedDict([('ImageName', polygon['file_name']),
                                       ('Subset', polygon['subset'])
                                       ]),
            'type': 'Feature'}
      c.write(record)

# Zip output folder for download

In [41]:
# zip up the output directory into an archive for download
output_file_name = 'Step_5_{o}'.format(o=output_dir)
import subprocess
subprocess.call(['zip', '-r', output_file_name + '.zip', '/content/' + output_dir])

from google.colab import files
files.download(output_file_name + ".zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>